In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from PIL import Image
import timm  # Import timm library
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
dd = pd.read_csv("./data/data_with_only_image_dir.csv")
dd.label.value_counts()

-1    7452
 1    2373
 0    1719
Name: label, dtype: int64

In [3]:
dd_filtered = dd[dd['label'] != 0]

# Sample equal number of -1 and 1 labels
dd_negative = dd_filtered[dd_filtered['label'] == -1].sample(n=2000, random_state=42)
dd_positive = dd_filtered[dd_filtered['label'] == 1].sample(n=2000, random_state=42)

# Concatenate the samples to create the final dataset
balanced_dataset = pd.concat([dd_negative, dd_positive])

# Shuffle the dataset
balanced_dataset = balanced_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

balanced_dataset['label'] = balanced_dataset['label'].replace(-1, 0)

balanced_dataset.to_csv('./data/balanced_dataset.csv', index=False)

In [4]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, data_length=None):
        # Assuming csv_file is a CSV file with columns "image_sample" and "label"
        self.data = pd.read_csv(csv_file)
        self.data = shuffle(self.data, random_state=42)
        if data_length != None:
            data_shape = self.data.shape
            included_data, excluded_data = train_test_split(self.data, test_size=1-(data_length/data_shape[0]), random_state=42, stratify=self.data.label)
            print(included_data.label.value_counts())
            self.data = included_data
#             self.data = self.data.iloc[:data_length]
            
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 0]  # assuming the image path is in the first column
        image = Image.open(img_path).convert('RGB')

        label = int(self.data.iloc[idx, 1])  # assuming the label is in the second column

        if self.transform:
            image = self.transform(image)

        return image, label

In [11]:
# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Split the dataset
dataset = CustomDataset(csv_file='./data/balanced_dataset.csv', transform=transform)
# train_size = int(0.8 * len(dataset))
# val_size = len(dataset) - train_size
# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


# Assuming you have your labels in the "label" column
labels = dataset.data['label'].values

# Use StratifiedShuffleSplit to split the dataset while maintaining class ratios
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, val_index in sss.split(range(len(dataset)), labels):
    train_dataset = torch.utils.data.Subset(dataset, train_index)
    val_dataset = torch.utils.data.Subset(dataset, val_index)
    
# Size of the training dataset
print("Size of the training dataset:", len(train_dataset))
print("Size of the testing dataset:", len(val_dataset))

# Optionally, you can convert train_dataset and val_dataset to DataLoader if needed
batch_size = 32


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

Size of the training dataset: 3200
Size of the testing dataset: 800


In [12]:
train_dataset.dataset.data.label.value_counts()

1    2000
0    2000
Name: label, dtype: int64

In [13]:
# Define the Vision Transformer Model
model = timm.create_model('vit_base_patch16_224', pretrained=True)

# Modify the Model Head
num_classes = 2
model.head = nn.Linear(model.head.in_features, num_classes)

### Stage 1 training (3 epochs)

In [14]:
# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

best_loss = float('inf')  # Initialize with a large value

# Training
num_epochs = 10
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

for epoch in tqdm(range(3)):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in tqdm(train_loader, desc='Epoch {}'.format(epoch), leave=False):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")
    
    # Save the model if it has the best loss so far
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), 'best_model_weights.pth')

Epoch 0:  95%|███████████████████████████████▎ | 95/100 [28:34<01:33, 18.67s/it]


Epoch 0: 100%|████████████████████████████████| 100/100 [30:08<00:00, 18.86s/it]
                                                                                

Epoch 1/3, Loss: 1.3809153044223785


Epoch 1:  95%|███████████████████████████████▎ | 95/100 [29:55<01:36, 19.25s/it]


Epoch 1: 100%|████████████████████████████████| 100/100 [31:31<00:00, 19.15s/it]
                                                                                

Epoch 2/3, Loss: 0.580397612452507


Epoch 2:  95%|███████████████████████████████▎ | 95/100 [32:08<01:42, 20.55s/it]


Epoch 2: 100%|████████████████████████████████| 100/100 [33:50<00:00, 20.43s/it]
                                                                                

Epoch 3/3, Loss: 0.5505284512042999


100%|█████████████████████████████████████████| 3/3 [1:35:33<00:00, 1911.31s/it]


In [19]:
torch.save(model.state_dict(), 'best_model_weights1.pth')

### Stage 2 training (3 epochs)

In [ ]:
model.load_state_dict(torch.load('best_model_weights1.pth'))

for epoch in tqdm(range(3, 6)):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in tqdm(train_loader, desc='Epoch {}'.format(epoch), leave=False):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")
    
    # Save the model if it has the best loss so far
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), 'best_model_weights.pth')

In [ ]:
torch.save(model.state_dict(), 'best_model_weights2.pth')

### Stage 3 training (4 epochs)

In [ ]:
model.load_state_dict(torch.load('best_model_weights2.pth'))

for epoch in tqdm(range(6, num_epochs)):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in tqdm(train_loader, desc='Epoch {}'.format(epoch), leave=False):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")
    
    # Save the model if it has the best loss so far
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), 'best_model_weights.pth')

In [ ]:
torch.save(model.state_dict(), 'best_model_weights3.pth')

In [15]:
print(best_loss)

0.5505284512042999


In [16]:
# Evaluation

# Load the saved weights
model.load_state_dict(torch.load('best_model_weights.pth'))

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(val_loader):

        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

100%|███████████████████████████████████████████| 25/25 [02:29<00:00,  5.96s/it]


In [18]:

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy * 100:.4f}%")

Test Accuracy: 74.7500%
